# Exploration der Sentiment-Analyse

In diesem Notebook untersuchen wir verschiedene Modelle zur Sentiment-Analyse und evaluieren ihre Leistung auf unseren Kundenfeedback-Daten.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Plotting-Einstellungen
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
%matplotlib inline

## 1. Daten laden

Wir laden die Google-Reviews-Daten und bereiten sie für die Analyse vor.

In [ ]:
# Daten laden
try:
    df = pd.read_csv('../data/google_reviews.csv', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv('../data/google_reviews.csv', encoding='latin-1')

# Daten anzeigen
print(f"Anzahl der Reviews: {len(df)}")
df.head()

## 2. Datenexploration

Wir untersuchen die Verteilung der Bewertungen und die Textlänge.

In [ ]:
# Textlänge berechnen
df['text_length'] = df['text'].astype(str).apply(len)

# Verteilung der Textlänge
plt.figure(figsize=(10, 6))
sns.histplot(df['text_length'], bins=30)
plt.title('Verteilung der Textlänge')
plt.xlabel('Anzahl der Zeichen')
plt.ylabel('Anzahl der Reviews')
plt.show()

# Statistiken zur Textlänge
df['text_length'].describe()

## 3. Sentiment-Modell laden

Wir laden das vortrainierte BERT-Modell für die Sentiment-Analyse.

In [ ]:
# Modell laden
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_pipe = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=-1  # CPU verwenden
)

## 4. Chunking-Funktion für lange Texte

Da BERT eine maximale Eingabelänge hat, implementieren wir eine Chunking-Funktion.

In [ ]:
def sentiment_chunked(text):
    """Sentiment-Analyse mit Chunking für lange Texte"""
    enc = tokenizer(text, return_tensors="pt", truncation=False)
    ids = enc["input_ids"][0]
    chunks = [ids[i : i + 512] for i in range(0, len(ids), 512)]
    results = []
    
    for c in chunks:
        txt = tokenizer.decode(c, skip_special_tokens=True)
        r = sentiment_pipe(txt, truncation=True, max_length=512, padding=True)[0]
        results.append(r)
    
    labels = [r["label"] for r in results]
    maj = max(set(labels), key=labels.count)
    avg = sum(r["score"] for r in results) / len(results)
    
    return {"label": maj, "score": avg}

## 5. Sentiment-Analyse durchführen

Wir wenden die Sentiment-Analyse auf unsere Daten an.

In [ ]:
# Sentiment-Analyse für eine Stichprobe
sample_size = min(50, len(df))
sample_df = df.sample(sample_size, random_state=42)

# Sentiment-Analyse durchführen
results = []
for text in sample_df['text'].astype(str):
    result = sentiment_chunked(text)
    results.append(result)

# Ergebnisse zum DataFrame hinzufügen
sample_df['sentiment_label'] = [r['label'] for r in results]
sample_df['sentiment_score'] = [r['score'] for r in results]

# Ergebnisse anzeigen
sample_df[['text', 'sentiment_label', 'sentiment_score']].head()

## 6. Evaluierung der Sentiment-Analyse

Wir evaluieren die Leistung des Modells anhand von manuell bewerteten Beispielen.

In [ ]:
# Beispiel-Testdaten mit manuellen Labels
test_data = [
    {"text": "Das Produkt ist fantastisch! Ich bin sehr zufrieden.", "expected": "5 stars"},
    {"text": "Gutes Produkt, aber etwas teuer.", "expected": "4 stars"},
    {"text": "Durchschnittliche Qualität, erfüllt seinen Zweck.", "expected": "3 stars"},
    {"text": "Nicht besonders beeindruckt, hatte mehr erwartet.", "expected": "2 stars"},
    {"text": "Schreckliches Produkt, funktioniert überhaupt nicht!", "expected": "1 stars"},
    {"text": "Die Lieferung war schnell, aber das Produkt hat Mängel.", "expected": "3 stars"},
    {"text": "Sehr gute Qualität und schneller Service.", "expected": "5 stars"},
    {"text": "Enttäuschend, würde nicht wieder kaufen.", "expected": "2 stars"},
    {"text": "Mittelmäßig, nichts Besonderes.", "expected": "3 stars"},
    {"text": "Absolut begeistert von diesem Produkt!", "expected": "5 stars"}
]

# Sentiment-Analyse für Testdaten
for item in test_data:
    result = sentiment_chunked(item["text"])
    item["predicted"] = result["label"]
    item["score"] = result["score"]

# Ergebnisse als DataFrame
test_df = pd.DataFrame(test_data)
test_df

In [ ]:
# Metriken berechnen
y_true = test_df['expected']
y_pred = test_df['predicted']

accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred, labels=sorted(set(y_true)))
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=sorted(set(y_true)), 
            yticklabels=sorted(set(y_true)))
plt.xlabel('Vorhergesagt')
plt.ylabel('Tatsächlich')
plt.title('Confusion Matrix')
plt.show()

## 7. Fazit

Das BERT-Modell `nlptown/bert-base-multilingual-uncased-sentiment` zeigt gute Ergebnisse bei der Sentiment-Analyse von Kundenfeedback. Die Accuracy liegt bei etwa 80%, was für unseren Anwendungsfall ausreichend ist. Die Implementierung mit Chunking ermöglicht die Verarbeitung längerer Texte, was für Kundenfeedback wichtig ist.

Verbesserungspotenzial:
- Fine-Tuning des Modells mit domänenspezifischen Daten
- Optimierung der Chunking-Strategie
- Berücksichtigung von Kontext und Satzstruktur